In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from scipy import integrate

# Constants of the system
gravitational_constant = 6.67e-11
mass_earth = 5.9736e24
mass_moon = 0.07349e24
distance_earth_moon = 3.844e8
angular_velocity = 2.6617e-6
radius_earth = 6.37816e6
radius_moon = 1.7374e6

def system_dynamics(coordinates, time):
    momentum_radial, momentum_angular, radial_distance, angular_position = coordinates
    delta = gravitational_constant * mass_earth / distance_earth_moon**3
    mu = mass_moon / mass_earth
    radial_prime = np.sqrt(1 + radial_distance**2 - 2*radial_distance*np.cos(angular_position - angular_velocity*time))

    drdt = momentum_radial
    dphidt = momentum_angular / radial_distance**2
    dPrdt = momentum_angular**2 / radial_distance**3 - delta * (1 / radial_distance**2 + (mu / radial_prime**3) * (radial_distance - np.cos(angular_position - angular_velocity*time)))
    dP_phidt = -delta * mu * radial_distance / (radial_prime**3) * np.sin(angular_position - angular_velocity*time)

    return np.array([dPrdt, dP_phidt, drdt, dphidt])

def runge_kutta(f, initial_conditions, time_points):
    time_step = time_points[1] - time_points[0]

    momentum_radial = np.zeros_like(time_points)
    momentum_angular = np.zeros_like(time_points)
    radial_distance = np.zeros_like(time_points)
    angular_position = np.zeros_like(time_points)

    momentum_radial[0], momentum_angular[0], radial_distance[0], angular_position[0] = initial_conditions

    for i in range(1, len(time_points)):
        current_state = np.array([momentum_radial[i-1], momentum_angular[i-1], radial_distance[i-1], angular_position[i-1]])
        current_time = time_points[i-1]

        k1 = f(current_state, current_time)
        k2 = f(current_state + 0.5*k1*time_step, current_time + 0.5*time_step)
        k3 = f(current_state + 0.5*k2*time_step, current_time + 0.5*time_step)
        k4 = f(current_state + k3*time_step, current_time + time_step)
        increment = (k1 + 2 * k2 + 2 * k3 + k4) * (time_step / 6)

        momentum_radial[i] = momentum_radial[i-1] + increment[0]
        momentum_angular[i] = momentum_angular[i-1] + increment[1]
        radial_distance[i] = radial_distance[i-1] + increment[2]
        angular_position[i] = angular_position[i-1] + increment[3]

    return np.array([momentum_radial, momentum_angular, radial_distance, angular_position])

time_points = np.arange(0., 1e6, 1)
initial_velocity = 11.1e3 / distance_earth_moon
initial_angular_position = 0 * np.pi / 180
initial_theta = 30 * np.pi / 180
initial_radial_distance = radius_earth / distance_earth_moon
momentum_radial, momentum_angular = (initial_velocity * np.cos(initial_theta - initial_angular_position), initial_velocity * initial_radial_distance * np.sin(initial_theta - initial_angular_position))
initial_conditions = [momentum_radial, momentum_angular, initial_radial_distance, initial_angular_position]
solution = runge_kutta(system_dynamics, initial_conditions, time_points)

# Set x and y coordinates of the moon and the satellite
x_moon = distance_earth_moon * np.cos(angular_velocity * time_points)
y_moon = distance_earth_moon * np.sin(angular_velocity * time_points)
x_satellite = solution[2] * np.cos(solution[3]) * distance_earth_moon
y_satellite = solution[2] * np.sin(solution[3]) * distance_earth_moon

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot()
scale = 1000
time_days = time_points[::scale] * (1 / 86400)

def initialize_plot():
    ax.clear()
    ax.set_xlim(-4e8, 4e8)
    ax.set_ylim(-4e8, 4e8)

def update_plot(i):
    initialize_plot()
    ax.scatter(x_moon[::scale][i], y_moon[::scale][i], color='k')
    ax.scatter(0, 0, color='b')
    ax.set_title("t = {:.4f} Earth days".format(time_days[i]))
    ax.scatter(x_satellite[::scale][i], y_satellite[::scale][i], color='red')

animation = anim.FuncAnimation(fig, update_plot, frames=len(time_points[::scale]), init_func=initialize_plot)
animation.save('MoonTrip.gif', writer='pillow')


 85%|████████▌ | 853582/999999 [06:21<01:12, 2022.64it/s]